In [6]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the data preprocessing
transform = transforms.Compose([
    transforms.Resize(224),  # Resize the image to 224x224 pixels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Download and load the CIFAR-10 training and test datasets
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torch.nn as nn
class AlexNet_Cus(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet_Cus, self).__init__()
        self.features = nn.Sequential(
            # conv1.
            nn.Conv2d(3, 96, 11, 4, 2),
            # inplace=True 的意思是直接在输入数据上进行修改，而不使用额外的内存来存储输出。
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(size=5, alpha=1e-4, beta=0.75, k=2),

            # conv2.
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(size=5, alpha=1e-4, beta=0.75, k=2),

            # conv3.
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(inplace=True),
            # conv4.
            nn.Conv2d(384, 384, 1, 1),
            nn.ReLU(inplace=True),
            # conv5.
            nn.Conv2d(384, 256, 1, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
            x = self.features(x)
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.classifier(x)
            return x

In [7]:
import torch.optim as optim
num_classes = 10
model = AlexNet_Cus(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [9]:
num_epochs = 2
for epoch in range(num_epochs):  # num_epochs是您想要运行的时期数量
    model.train()  # 将模型设置为训练模式
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()  # 清空梯度
        outputs = model(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新权重

        if (i + 1) % 100 == 0:  # 每100次迭代打印一次状态
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/2], Step [100/1563], Loss: 2.2993
Epoch [1/2], Step [200/1563], Loss: 2.2991
Epoch [1/2], Step [300/1563], Loss: 2.3029
Epoch [1/2], Step [400/1563], Loss: 2.3012
Epoch [1/2], Step [500/1563], Loss: 2.3016
Epoch [1/2], Step [600/1563], Loss: 2.3007
Epoch [1/2], Step [700/1563], Loss: 2.3022
Epoch [1/2], Step [800/1563], Loss: 2.3033
Epoch [1/2], Step [900/1563], Loss: 2.3068
Epoch [1/2], Step [1000/1563], Loss: 2.3012
Epoch [1/2], Step [1100/1563], Loss: 2.3013
Epoch [1/2], Step [1200/1563], Loss: 2.3020
Epoch [1/2], Step [1300/1563], Loss: 2.3009
Epoch [1/2], Step [1400/1563], Loss: 2.3019
Epoch [1/2], Step [1500/1563], Loss: 2.3012
Epoch [2/2], Step [100/1563], Loss: 2.2987
Epoch [2/2], Step [200/1563], Loss: 2.3014
Epoch [2/2], Step [300/1563], Loss: 2.2969
Epoch [2/2], Step [400/1563], Loss: 2.2902
Epoch [2/2], Step [500/1563], Loss: 2.2997
Epoch [2/2], Step [600/1563], Loss: 2.2918
Epoch [2/2], Step [700/1563], Loss: 2.2448
Epoch [2/2], Step [800/1563], Loss: 2.1714
Epoch

In [12]:
model.eval()  # 将模型设置为评估模式
with torch.no_grad():  # 关闭梯度计算，节省内存和计算资源
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # 确保将数据移到CUDA上
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # 不需要再次将输出移到cpu
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))


Accuracy of the network on the test images: 31 %
